In [161]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import math
from sklearn.multioutput import MultiOutputRegressor
import matplotlib.pyplot as plt # 득점모델 변수 중요도
import xgboost


import pandas as pd
import random
import os
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import MaxAbsScaler
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.model_selection import KFold
%matplotlib inline

In [141]:
pd.set_option('display.max_columns', None)

In [142]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(42) # Seed 고정

In [143]:
##train데이터 불러오기
train_df = pd.read_csv('./train.csv')
test_df = pd.read_csv('./test.csv')

In [144]:
train_df.drop([ 'X_48', 'X_47', 'X_23', 'X_11', 'X_10', 'X_04'], inplace= True, axis=1)
test_df.drop([ 'X_48', 'X_47', 'X_23', 'X_11', 'X_10', 'X_04'], inplace= True, axis=1)

In [145]:
train_x = train_df.filter(regex='X') # Input : X Featrue
train_y = train_df.filter(regex='Y') # Output : Y_01 Feature..
test_x = test_df.filter(regex='X')
test_y = test_df.filter(regex='Y')

In [146]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=5, init='k-means++', max_iter=300, random_state=0) # K-means
kmeans.fit(train_x)

KMeans(n_clusters=5, random_state=0)

In [147]:
kmm_cluster_labels_tr = kmeans.predict(train_x)
kmm_cluster_labels_te = kmeans.predict(test_x)

In [148]:
train_x["km_label"] = kmm_cluster_labels_tr
test_x["km_label"] = kmm_cluster_labels_te

C:\Users\User\AppData\Local\Temp\ipykernel_16264\641694845.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_x["km_label"] = kmm_cluster_labels_tr


## PCA

In [149]:
pca_list = [['X_01','X_02','X_05','X_06'],['X_14','X_15','X_16','X_17','X_18'],['X_19','X_20','X_21','X_22'],['X_30','X_31','X_32','X_33'],['X_24','X_25','X_26','X_27','X_28','X_29'],['X_50','X_51','X_52','X_53','X_54','X_55','X_56']]

In [152]:
from sklearn.preprocessing import MaxAbsScaler
train_col = train_x.columns
train_x = MaxAbsScaler().fit_transform(train_x)
train_x = pd.DataFrame(data = train_x, columns=train_col)

In [153]:
pca = PCA(n_components=1)
cnt = 1
pca_result = []
for i in pca_list:
    pca_x = train_x[i]
    printcipalComponents = pca.fit_transform(pca_x)
    pca_result.append(printcipalComponents)
    cnt += 1

In [154]:
cnt =1
for i in pca_result:
    train_x[f"{cnt}_X_comp"] = i
    cnt += 1

In [155]:
for i in pca_list:
    train_x.drop(i, inplace = True, axis=1)

In [156]:
train_x

,X_03,X_07,X_08,X_09,X_12,X_13,X_34,X_35,X_36,X_37,X_38,X_39,X_40,X_41,X_42,X_43,X_44,X_45,X_46,X_49,km_label,1_X_comp,2_X_comp,3_X_comp,4_X_comp,5_X_comp,6_X_comp
0,0.756645,0.179727,0.026128,0.385433,0.966592,0.642857,0.993119,0.983957,0.984721,0.993119,-0.509153,-0.957285,-0.961061,0.980574,0.979011,0.993928,0.989212,0.690476,0.995916,0.084722,1.00,0.049706,0.004100,-0.048706,0.068281,-0.000818,-0.045718
1,0.730851,0.175333,0.025647,0.366453,0.975501,0.642857,0.987768,0.983193,0.984721,0.988532,-0.498294,-0.942657,-0.944695,0.978723,0.980877,0.988323,0.991088,0.309524,0.995916,0.090984,1.00,0.026225,0.004316,-0.087367,0.060577,0.032691,-0.034541
2,0.718515,0.175821,0.044303,0.426987,0.971047,0.535714,0.991590,0.983193,0.983193,0.993884,-0.501396,-0.946167,-0.945824,0.977336,0.980877,0.988790,0.990619,0.333333,0.999319,0.095567,1.00,0.055924,0.004721,-0.083788,0.064730,-0.010557,-0.017229
3,0.757766,0.176492,0.048257,0.400571,0.964365,0.750000,0.987003,0.990833,0.992361,0.987768,-0.497983,-0.937975,-0.940745,0.979648,0.978545,0.990191,0.989212,0.523810,1.000000,0.130993,0.25,0.073251,0.008751,-0.090252,0.060200,-0.051817,-0.069514
4,0.712908,0.181130,0.043302,0.378767,0.968820,0.571429,0.990826,0.981665,0.986249,0.990826,-0.504189,-0.949678,-0.950903,0.978723,0.977612,0.989257,0.989681,0.523810,1.000000,0.096462,1.00,0.012401,0.005503,-0.081160,0.074656,0.026664,-0.034411
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39602,0.698329,0.184304,0.032600,0.467537,0.971047,0.535714,0.985474,0.993125,0.988541,0.986239,-0.499224,-0.944997,-0.947517,0.979186,0.983209,0.989724,0.993902,0.261905,1.000000,0.529232,0.50,-0.043568,-0.002210,-0.055259,0.002287,0.002174,0.043025
39603,0.703936,0.178262,0.042828,0.424587,0.979955,0.464286,0.984709,0.983957,0.984721,0.983180,-0.487124,-0.921006,-0.925508,0.978723,0.983675,0.990659,0.993902,0.285714,0.992512,0.530390,0.50,-0.044246,0.001723,-0.068477,0.023085,0.011815,0.038635
39604,0.725244,0.182839,0.042979,0.310703,0.975501,0.500000,0.992355,0.991597,0.991597,0.979358,-0.500155,-0.947338,-0.948081,0.979186,0.983675,0.991126,0.993902,0.309524,0.993193,0.076930,1.00,-0.010414,-0.001309,-0.043088,0.005659,-0.007737,0.003487
39605,0.714029,0.184914,0.047163,0.432195,0.964365,0.571429,0.987003,0.985485,0.990069,0.991590,-0.507602,-0.957285,-0.957675,0.978261,0.981810,0.988323,0.991088,0.261905,1.000000,0.543125,0.50,-0.037317,0.000460,-0.053361,0.000516,-0.045697,0.025362


In [160]:
xgb_model = xgboost.XGBRegressor(n_estimators=1000, max_depth=8, learning_rate=0.01, n_jobs=-1, gamma= 0.2)
XGB_Ro = MultiOutputRegressor(xgb_model).fit(train_x, train_y)

In [163]:
train_predict = XGB_Ro.predict(train_x)
print("RMSE':{}".format(math.sqrt(mean_squared_error(train_predict, train_y))))

RMSE':0.9875602094019984


In [167]:
pd.DataFrame(train_predict,columns = train_y.columns).describe()

,Y_01,Y_02,Y_03,Y_04,Y_05,Y_06,Y_07,Y_08,Y_09,Y_10,Y_11,Y_12,Y_13,Y_14
count,39607.000000,39607.000000,39607.000000,39607.000000,39607.000000,39607.000000,39607.000000,39607.000000,39607.000000,39607.000000,39607.000000,39607.000000,39607.000000,39607.000000
mean,1.353738,1.057186,1.013930,13.620820,31.289715,16.529505,3.154878,-26.293493,-26.307240,-22.398647,24.324202,-26.236397,-26.232510,-26.244503
std,0.111639,0.113362,0.099489,0.914789,0.647926,1.250627,0.121274,0.218666,0.211159,0.374380,0.212436,0.213067,0.213664,0.213853
min,0.274870,0.376155,0.264732,8.518718,24.420166,-18.060017,1.302985,-27.663937,-27.823275,-29.113297,21.875595,-27.926640,-27.891256,-27.935921
25%,1.288805,0.990806,0.955835,13.112654,30.994278,16.503779,3.089638,-26.436066,-26.440026,-22.511211,24.219540,-26.369656,-26.366773,-26.377220
50%,1.357230,1.060588,1.018527,13.559438,31.331554,16.593073,3.163359,-26.282793,-26.303246,-22.322666,24.321226,-26.229940,-26.227076,-26.237484
75%,1.422984,1.124801,1.073437,14.070251,31.650058,16.691635,3.219131,-26.170855,-26.194941,-22.198818,24.428722,-26.120354,-26.116181,-26.127979
max,2.703909,2.642728,2.150704,79.915024,34.676643,17.813843,4.223561,-24.469006,-24.792166,-20.797062,25.754976,-24.635752,-24.555773,-24.356871


In [170]:
loss_fold = []
kfold = KFold(n_splits=3, shuffle=True)
for train, test in kfold.split(train_x):
    XGB = MultiOutputRegressor(xgb_model).fit(train_x.loc[train], train_y.loc[train])
    train_predict = XGB.predict(train_x.loc[test])
    scores = math.sqrt(mean_squared_error(train_predict, train_y.loc[test]))
    loss_fold.append(scores)

In [171]:
loss_fold

[1.1968468203154572, 1.2109570822549804, 1.1965353319503536]